In [14]:
# importando libs
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import os
from datetime import datetime
from datetime import timedelta
import yfinance as yf
import pickle
from prophet import Prophet
from prophet.serialize import model_to_json, model_from_json
from prophet import plot
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [15]:
DAYS_PREDICT = 15

In [16]:
date_today = datetime.today().strftime("%Y-%m-%d")
data_init = "2014-01-01"

df_xrp = yf.download("XRP-USD", data_init, date_today)

df_xrp.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-11-09,0.217911,0.221791,0.214866,0.217488,0.217488,147916992
2017-11-10,0.218256,0.219068,0.205260,0.206483,0.206483,141032992
2017-11-11,0.205948,0.214456,0.205459,0.210430,0.210430,134503008
2017-11-12,0.210214,0.210214,0.195389,0.197339,0.197339,251175008
2017-11-13,0.197472,0.204081,0.197456,0.203442,0.203442,132567000


In [17]:
df_xrp.reset_index(inplace=True)
df_xrp.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
1768,2022-09-12,0.355145,0.359445,0.349269,0.359445,0.359445,1077587297
1769,2022-09-13,0.359464,0.359643,0.332044,0.333692,0.333692,1443625685
1770,2022-09-14,0.332960,0.343893,0.332426,0.342082,0.342082,1227835854
1771,2022-09-15,0.342075,0.342461,0.325600,0.326335,0.326335,1521572444
1772,2022-09-16,0.326384,0.357134,0.323541,0.356688,0.356688,1375977237


In [18]:
df = df_xrp[["Date", "Adj Close"]]
df.rename(columns= { "Date": "ds", "Adj Close": "y" }, inplace=True)

In [19]:
df.tail()

,ds,y
1768,2022-09-12,0.359445
1769,2022-09-13,0.333692
1770,2022-09-14,0.342082
1771,2022-09-15,0.326335
1772,2022-09-16,0.356688


In [20]:
assert df.isnull().sum().sum() == 0
assert df['ds'].isnull().sum() == 0
assert df['y'].isnull().sum() == 0

In [21]:
df.dtypes

ds    datetime64[ns]
y            float64
dtype: object

In [22]:
model = Prophet(seasonality_mode="multiplicative")
model.fit(df)

00:34:46 - cmdstanpy - INFO - Chain [1] start processing
00:34:46 - cmdstanpy - INFO - Chain [1] done processing


In [23]:
df_future = model.make_future_dataframe(periods=DAYS_PREDICT)
df_future.tail()

,ds
1783,2022-09-27
1784,2022-09-28
1785,2022-09-29
1786,2022-09-30
1787,2022-10-01


In [24]:
# predict 
predict = model.predict(df_future)
predict

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
0,2017-11-09,1.183318,0.735231,1.224809,1.183318,1.183318,-0.176668,-0.176668,-0.176668,-0.002956,-0.002956,-0.002956,-0.173712,-0.173712,-0.173712,0.0,0.0,0.0,0.974263
1,2017-11-10,1.182121,0.700731,1.199763,1.182121,1.182121,-0.193965,-0.193965,-0.193965,0.002017,0.002017,0.002017,-0.195982,-0.195982,-0.195982,0.0,0.0,0.0,0.952831
2,2017-11-11,1.180924,0.701910,1.171463,1.180924,1.180924,-0.207331,-0.207331,-0.207331,0.009510,0.009510,0.009510,-0.216841,-0.216841,-0.216841,0.0,0.0,0.0,0.936082
3,2017-11-12,1.179727,0.669092,1.159855,1.179727,1.179727,-0.228910,-0.228910,-0.228910,0.007309,0.007309,0.007309,-0.236219,-0.236219,-0.236219,0.0,0.0,0.0,0.909675
4,2017-11-13,1.178530,0.614999,1.117603,1.178530,1.178530,-0.256009,-0.256009,-0.256009,-0.001921,-0.001921,-0.001921,-0.254088,-0.254088,-0.254088,0.0,0.0,0.0,0.876816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1783,2022-09-27,0.224894,-0.035868,0.463180,0.224894,0.224894,-0.052433,-0.052433,-0.052433,-0.010347,-0.010347,-0.010347,-0.042086,-0.042086,-0.042086,0.0,0.0,0.0,0.213102
1784,2022-09-28,0.222707,-0.029367,0.442438,0.222707,0.222707,-0.045878,-0.045878,-0.045878,-0.003612,-0.003612,-0.003612,-0.042266,-0.042266,-0.042266,0.0,0.0,0.0,0.212490
1785,2022-09-29,0.220520,-0.044380,0.466948,0.220520,0.220520,-0.043857,-0.043857,-0.043857,-0.002956,-0.002956,-0.002956,-0.040901,-0.040901,-0.040901,0.0,0.0,0.0,0.210848
1786,2022-09-30,0.218333,-0.044647,0.444053,0.218333,0.218333,-0.035881,-0.035881,-0.035881,0.002017,0.002017,0.002017,-0.037898,-0.037898,-0.037898,0.0,0.0,0.0,0.210499


In [25]:
predict[["ds", "yhat_lower", "yhat_upper", "yhat"]].tail()

,ds,yhat_lower,yhat_upper,yhat
1783,2022-09-27,-0.035868,0.463180,0.213102
1784,2022-09-28,-0.029367,0.442438,0.212490
1785,2022-09-29,-0.044380,0.466948,0.210848
1786,2022-09-30,-0.044647,0.444053,0.210499
1787,2022-10-01,-0.042198,0.469074,0.211026


In [26]:
# Salvar modelo
with open('model_xrp_usd.pckl', 'wb') as fout:
    pickle.dump(model, fout)